In [2]:
from common import *
import xml.etree.ElementTree as ET

Using TensorFlow backend.


In [9]:
fname = join(join(DATA_FOLDER, 'rawdata'), 'all_docs_raw.txt')

In [10]:
# all_docs = glob(join(DATA_FOLDER, 'rawdata') + '/**/*.xml', recursive=True)
# all_docs = sorted(all_docs)
# with open(fname, mode='wt', encoding='utf-8') as f:
#     f.write('\n'.join(all_docs))

In [11]:
with open(fname, mode='r', encoding='utf-8') as f:
    all_docs = f.read().splitlines()

In [12]:
len(all_docs)

1266990

In [329]:
afile = '../data/rawdata/2016/RU/A/20161227/2015116494/2015116494.xml'
c1file = '../data/rawdata/2016/RU/C1/20161227/0002605713/0002605713.xml'
c2file = '../data/rawdata/2016/RU/C2/20160127/0002573901/0002573901.xml'
ufile = '../data/rawdata/2016/RU/U1/20161210/0000166582/0000166582.xml'

In [330]:
def get_node(root, name, lang='ru'):
    for node in root.iter(name):
        if node.attrib['lang'].lower() == lang:
            return node

In [337]:
doc = {}

tree = ET.parse(c2file)
root = tree.getroot()

doc.update(root.attrib)

SDOBI = get_node(root, 'SDOBI')
name = SDOBI.find('./B500/ru-b540/ru-b542').text.lower()
doc['name_raw'] = name

node = get_node(root, 'abstract')
if node is not None:
    doc['abstract_raw'] = ' '.join(node.itertext()).lower()

node = SDOBI.find('./B500/B560/ru-b560')
if node is not None:
    doc['ru-b560'] = ' '.join(node.itertext()).lower()

node = get_node(root, 'description')
if node is not None:
    doc['description_raw'] = ' '.join(node.itertext()).lower()

node = get_node(root, 'claims')
if node is not None:
    doc['claims_raw'] = ' '.join(node.itertext()).lower()
            

In [342]:
print(doc['abstract_raw'])

изобретение относится к способу получения поверхностного покрытия, содержащему этапы обеспечения слоя износа на основе пвх (1), по существу, свободного от неорганических наполнителей, нанесения на упомянутый слой на основе пвх (1) композиции верхнего покрытия, содержащей поливиниловый спирт (пвс) и соединение силана, содержащее по меньшей мере одну аминогруппу, нагревание упомянутого слоя на основе пвх (1) и упомянутой композиции верхнего покрытия при температуре, эквивалентной или превосходящей 150°c, для образования верхнего покрытия слоя износа (2). результатом является получение поверхностного покрытия, сочетающего свойства физической устойчивости и свойства химического и газового барьера. 2 н. и 12 з.п. ф-лы, 4 ил., 6 табл.


In [338]:
doc

{'abstract_raw': 'изобретение относится к способу получения поверхностного покрытия, содержащему этапы обеспечения слоя износа на основе пвх (1), по существу, свободного от неорганических наполнителей, нанесения на упомянутый слой на основе пвх (1) композиции верхнего покрытия, содержащей поливиниловый спирт (пвс) и соединение силана, содержащее по меньшей мере одну аминогруппу, нагревание упомянутого слоя на основе пвх (1) и упомянутой композиции верхнего покрытия при температуре, эквивалентной или превосходящей 150°c, для образования верхнего покрытия слоя износа (2). результатом является получение поверхностного покрытия, сочетающего свойства физической устойчивости и свойства химического и газового барьера. 2 н. и 12 з.п. ф-лы, 4 ил., 6 табл.',
 'bullid': 'd03',
 'claims_raw': '1. способ получения поверхностного покрытия на основе поливинилхлорида (пвх), включающий следующие этапы: - обеспечение слоя износа на основе пвх (1), причем количество содержащихся неорганических наполнител